In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Atividade2").getOrCreate()
volumes_path = "/Volumes/atividade_2_rocketlab/landing/volumes/"
cs = spark.read.csv(volumes_path + 'olist_customers_dataset.csv', header=True, inferSchema=True)
geo = spark.read.csv(volumes_path + 'olist_geolocation_dataset.csv', header=True, inferSchema=True)
ord_items = spark.read.csv(volumes_path + 'olist_order_items_dataset.csv', header=True, inferSchema=True)
ord_pay = spark.read.csv(volumes_path + 'olist_order_payments_dataset.csv', header=True, inferSchema=True)
ord_reviews = spark.read.csv(volumes_path + 'olist_order_reviews_dataset.csv', header=True, inferSchema=True) 
orders = spark.read.csv(volumes_path + 'olist_orders_dataset.csv', header=True, inferSchema=True)
sellers = spark.read.csv(volumes_path + 'olist_sellers_dataset.csv', header=True, inferSchema=True)
prod_catg_translation = spark.read.csv(volumes_path + 'products_category_name_translation.csv', header=True, inferSchema=True)